In [1]:
%load_ext watermark
%watermark -a 'orii-blackmark' -v -p torch

Author: NavinKumarMNK

Python implementation: CPython
Python version       : 3.10.5
IPython version      : 8.22.2

torch: 2.2.1



In [2]:
# Import the required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchvision import models
import wandb
import torchmetrics


In [3]:
#model = models.efficientnet_v2_m(include_top=False, weights='EfficientNet_V2_M_Weights.DEFAULT')

#model = models.efficientnet_b3(include_top=False, weights='EfficientNet_B3_Weights.DEFAULT')
# remove last layer
#model.classifier = nn.Identity()

# Add the parent directory to the path
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../')))

from models.EfficientNetv2.EncoderCoAtNet import EncoderCoAtNet
model = EncoderCoAtNet(num_blocks=[2, 2, 6, 14, 2])
model.summarize()

OSError: [WinError 127] The specified procedure could not be found

In [ ]:
torch.save(model, '../weights/EfficientNetv2Encoder.pt')

In [ ]:
model_1 = torch.load('../weights/EfficientNetv2Encoder.pt')

In [ ]:
import torch.nn as nn

class SEAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y
    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        # Initial representation
        self.fc = nn.Linear(1024, 2*2*1024)
        self.bn1d = nn.BatchNorm1d(2*2*1024)
        self.gelu = nn.GELU()

        # Decoder layers
            
        self.conv1 = nn.ConvTranspose2d(1024, 768, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn1 = nn.BatchNorm2d(768)
        self.relu1 = nn.GELU()

        self.conv9 = nn.ConvTranspose2d(768, 512, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn9 = nn.BatchNorm2d(512)
        self.relu9 = nn.GELU()

        self.conv12 = nn.ConvTranspose2d(512, 512, kernel_size=5, stride=1, padding=2, output_padding=0)
        self.bn12 = nn.BatchNorm2d(512)
        self.relu12 = nn.GELU()


        self.conv11 = nn.ConvTranspose2d(512, 384, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn11 = nn.BatchNorm2d(384)
        self.relu11 = nn.GELU()

        self.conv2 = nn.ConvTranspose2d(384, 256, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn2 = nn.BatchNorm2d(256)
        self.relu2 = nn.GELU()

        self.conv10 = nn.ConvTranspose2d(256, 256, kernel_size=5, stride=1, padding=2, output_padding=0)
        self.bn10 = nn.BatchNorm2d(256)
        self.relu10 = nn.GELU()

        self.conv3 = nn.ConvTranspose2d(256, 192, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn3 = nn.BatchNorm2d(192)
        self.relu3 = nn.GELU()
        
        self.conv4 = nn.ConvTranspose2d(192, 128, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.GELU()

        self.conv5 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, output_padding=0)
        self.bn5 = nn.BatchNorm2d(64)
        self.relu5 = nn.GELU()

        self.conv6 = nn.ConvTranspose2d(64, 64, kernel_size=5, stride=1, padding=2, output_padding=0)
        self.bn6 = nn.BatchNorm2d(64)
        self.relu6 = nn.GELU()

        self.conv7 = nn.ConvTranspose2d(64, 32, kernel_size=5, stride=1, padding=2, output_padding=0)
        self.bn7 = nn.BatchNorm2d(32)
        self.relu7 = nn.GELU()

        # Residual blocks with SE attention
        self.res1 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.Sigmoid(),
            SEAttention(256),
            nn.ReLU()
        )

        self.res2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.Sigmoid(),
            SEAttention(64),
            nn.ReLU()
        )

        self.dropout = nn.Dropout(0.25)
        
        self.conv8 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc(x)
        x = self.bn1d(x)
        x = self.gelu(x)

        x = x.view(x.size(0), 1024, 2, 2)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv9(x)
        x = self.bn9(x)
        x = self.relu9(x)

        x = self.conv12(x)
        x = self.bn12(x)
        x = self.relu12(x)
        x = self.dropout(x)

        x = self.conv11(x)
        x = self.bn11(x)
        x = self.relu11(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout(x)

        x = self.conv10(x)
        x = self.bn10(x)
        x = self.relu10(x)

        x = self.res1(x) + x
        x = self.dropout(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.dropout(x)

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)

        x = self.res2(x) + x
        x = self.dropout(x)

        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu7(x)

        x = self.conv8(x)

        x = self.tanh(x)

        return x


In [ ]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input = torch.randn(16, 3, 256, 256)
#encoder = model
decoder = Decoder()

#feature = encoder(input)
#print(feature.shape)
output = decoder(torch.randn(16, 1024))
print(output.shape)
# differnece between input and output

model2 = decoder
total_params = sum(p.numel() for p in model2.parameters())
print(total_params)


torch.Size([16, 3, 256, 256])
38730563


In [ ]:
torch.save(model.state_dict(), '../weights/EfficientNetv2DecoderLarge.pt', )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 torch.save(model.state_dict(), '../weights/EfficientNetv2DecoderLarge.pt', )                 │
│   2                                                                                              │
│                                                                                                  │
│ /home/windows/miniconda3/envs/ray/lib/python3.10/site-packages/torch/serialization.py:422 in     │
│ save                                                                                             │
│                                                                                                  │
│    419 │   _check_dill_version(pickle_module)                                                    │
│    420 │                                                                                         │
│    421 │   if _use_new_zipfile_serialization:                                                    │
│ ❱  422 │   │   with _open_zipfile_writer(f) as opened_zipfile:                                   │
│    423 │   │   │   _save(obj, opened_zipfile, pickle_module, pickle_protocol)                    │
│    424 │   │   │   return                                                                        │
│    425 │   else:                                                                                 │
│                                                                                                  │
│ /home/windows/miniconda3/envs/ray/lib/python3.10/site-packages/torch/serialization.py:290 in     │
│ __exit__                                                                                         │
│                                                                                                  │
│    287 │   │   super(_open_zipfile_writer_file, self).__init__(torch._C.PyTorchFileWriter(str(n  │
│    288 │                                                                                         │
│    289 │   def __exit__(self, *args) -> None:                                                    │
│ ❱  290 │   │   self.file_like.write_end_of_file()                                                │
│    291                                                                                           │
│    292                                                                                           │
│    293 class _open_zipfile_writer_buffer(_opener):                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
# classifier
class EfficientNetv2Classifier(nn.Module):
    def __init__(self, no_of_classes):
        super(EfficientNetv2Classifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, no_of_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.model(x)


In [ ]:
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))
from utils import utils

In [ ]:
classes = int(utils.config_parse('GENERAL')['no_of_classes'])
classes

In [ ]:

model = EfficientNetv2Classifier(classes)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

In [ ]:
torch.save(model.state_dict(), '../weights/EfficientNetv2Classifier.pt')